# Loós András Péter OXNGC0 : Projekt Graduation 2

## A Projekt célja:

Az elmúlt hetekben West úr politikailag nem helyes szólásai miatt szeretném átélni újra a régi szép időket, ezért a Graduation című albumának mintájára generálunk egy új számot.

#### Használt csomagok:

Numpy: matematikai függvénykönyvtár, többdimenziós tömbök és mátrixok használatára van

Tensorflow: Gépi tanulás és mesterséges intelligencia eszköz, a szöveg értelmezésére használjuk

Tqdm: Folyamat vizualizációs eszköz

In [1]:
#pip install tqdm
#pip install tensorflow
#pip install numpy

##### beolvassuk a szöveget és kisbetüssé tesszük.

In [2]:
with open("Graduation.txt", "r", encoding="utf8") as lyrics:
    script = lyrics.read().lower()

##### kigyűjtjük a használt karaktereket a szövegből

In [3]:
import numpy as np
from tqdm import tqdm

maxlen = 60
step = 3
sentences = []
next_chars = []

for i in tqdm(range(0, len(script) - maxlen, step)):
    sentences.append(script[i: i + maxlen])
    next_chars.append(script[i + maxlen])
len(sentences)

100%|██████████| 11789/11789 [00:00<00:00, 2357296.43it/s]


11789

##### a használt karaktereket és indexeiket eltároljuk

In [4]:
chars = sorted(list(set(script)))
char_indices = dict((char, chars.index(char)) for char in chars)

In [5]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)
for i, sentence in tqdm(enumerate(sentences)):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print("x", x.shape)
print("y", y.shape)

11789it [00:00, 92080.97it/s]

x (11789, 60, 45)
y (11789, 45)


##### elkezdjük a modell betanítását

25 epochot használok. Lehet hogy sok, de hátha értelmesebb lesz tőle a szöveg.

In [6]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.GRU(16, input_shape=(maxlen, len(chars))),
    tf.keras.layers.Dense(units=len(chars), activation='softmax')
]) 

model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 16)                3024      
                                                                 
 dense (Dense)               (None, 45)                765       
                                                                 
Total params: 3,789
Trainable params: 3,789
Non-trainable params: 0
_________________________________________________________________


In [7]:
with tf.device("/CPU:0"):
    model.fit(x, 
            y,
            batch_size=128,
            epochs=25)

Epoch 1/25
93/93 [==============================] - 2s 9ms/step - loss: 2.9847
Epoch 2/25
93/93 [==============================] - 1s 9ms/step - loss: 2.5105
Epoch 3/25
93/93 [==============================] - 1s 9ms/step - loss: 2.3426
Epoch 4/25
93/93 [==============================] - 1s 9ms/step - loss: 2.2365
Epoch 5/25
93/93 [==============================] - 1s 9ms/step - loss: 2.1660
Epoch 6/25
93/93 [==============================] - 1s 9ms/step - loss: 2.1131
Epoch 7/25
93/93 [==============================] - 1s 9ms/step - loss: 2.0639
Epoch 8/25
93/93 [==============================] - 1s 9ms/step - loss: 2.0252
Epoch 9/25
93/93 [==============================] - 1s 9ms/step - loss: 1.9941
Epoch 10/25
93/93 [==============================] - 1s 9ms/step - loss: 1.9628
Epoch 11/25
93/93 [==============================] - 1s 9ms/step - loss: 1.9412
Epoch 12/25
93/93 [==============================] - 1s 9ms/step - loss: 1.9252
Epoch 13/25
93/93 [==============================

##### fogunk egy véletlen sort. minden próba ezzel fog kezdeni

In [8]:
import random

start_index = random.randint(0, len(script) - maxlen - 1)
base_text = script[start_index: start_index + maxlen]
base_text

'tand there in a speedo\nand be looked at like a fucking hero\n'

In [9]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

##### kiírjuk a szöveget

megadjuk a generált szöveg hosszúságát, és hogy mennyire legyen magabiztos a modell

a 0.1 és 0.2 inkább vizualizációnak van ott, ha túl szorosan követi a gép a betanult szöveget akkor nem lesz értelmes annyira az eredmény.

In [10]:
import sys
temperatures = [0.1, 0.2, 0.5, 1.0]
generated_characters = 400

for temp in temperatures:
    print("Temp: ", temp)
    generated_text = base_text
    print(generated_text)
    for i in range(generated_characters):
        sampled = np.zeros((1, maxlen, len(chars)))    
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.          
        
        preds = model.predict(sampled, verbose=0)[0]      
        
        next_index = sample(preds, temp)
        next_char = chars[next_index]

        generated_text += next_char
        generated_text = generated_text[1:]

        sys.stdout.write(next_char)
        sys.stdout.flush()

    print()

Temp:  0.1
tand there in a speedo
and be looked at like a fucking hero

i shin't the shin't the shin't the shin't the shin't the shin't the that the i shind i shin't the so shin't the shin't the to i shing the shin't the shind the shind i shind i shin't the shin't the shin't the shong the i shin't the shin't the shind i cand i shind i shing the shon't the shin't the shin't the shin't the shin't the i shin't the shin't the here the i shin't the shin't the shin't the sh
Temp:  0.2
tand there in a speedo
and be looked at like a fucking hero

we her i'm i but the shon't the i so i shin't the to i shin't the now i'm in the shin't i'm and the i was the shong the i'm i can't the good to i can't the here the that the that the the here the i was the shin't the shind the to i shin't the i so stornger
you her hood the shin't the that the i'm in the the win't whet the hat to and i can't the the i shind i'm never in the gon't the shong the whet
Temp:  0.5
tand there in a speedo
and be looked at lik